In [206]:
import pandas as pd
import json
priceData='priceData/compound_price_data.csv'
jsonFile = 'sentimentalOutput/discord-compound.json'
def calculateAccuracy(priceData,jsonFile):
    monthlyPrice = pd.read_csv(priceData)
    if monthlyPrice['Price'].dtypes == 'object':
        newPrice = []
        for row in monthlyPrice.iterrows():
            price = row[1][2]
            if ',' in price:
                price = price.split(',')
                price = ''.join(price)
                price = float(price)
                newPrice.append(price)
            else:
                price = float(price)
                newPrice.append(price)
        monthlyPrice['Price'] = newPrice
    monthlyPrice = monthlyPrice[['Date','Price']]
    monthlyPrice['Date'] = pd.to_datetime(monthlyPrice['Date']).dt.to_period('M')
    monthlyPrice = monthlyPrice.groupby(pd.Grouper(key='Date', axis=0)).max().reset_index()
    monthlyPrice

    with open(jsonFile) as json_file:
        data = json.load(json_file)
    date = []
    senti = []
    for key in data:
        date.append(key)
        senti.append(data[key])
    s1={'Date':date,'sentiment':senti}
    s1 = pd.DataFrame(s1)
    s1["Date"] = pd.to_datetime(s1['Date']).dt.to_period('M')
    new = s1.merge(monthlyPrice, on='Date')
    new['changePrice'] = new['Price'].pct_change(fill_method ='ffill')
    new['changeSen'] = new['sentiment'].pct_change(fill_method ='ffill')
    new['changeSen'] = new['changeSen'].shift(periods=1)
    compare = new[['changeSen','changePrice']]
    compare = compare.dropna()
    compare['changeSen'] = [True if x > 0 else False for x in compare['changeSen']]
    compare['changePrice'] = [True if x > 0 else False for x in compare['changePrice']]
    compare['res'] = compare['changePrice'] == compare['changeSen']
    accuracy = compare['res'].mean()
    return accuracy
print(calculateAccuracy(priceData,jsonFile))

0.38461538461538464
